# <a href="https://miptstats.github.io/courses/ad_fivt.html">Введение в анализ данных</a>
## Домашнее задание 6. Основы обработки текстов


**Правила, <font color="red">прочитайте внимательно</font>:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_ds24_bot`. Для начала работы с ботом каждый раз отправляйте `/start`. **Работы, присланные иным способом, не принимаются.**
* Дедлайн см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно **ноутбук в формате `ipynb`**.
* Следите за размером файлов. **Бот не может принимать файлы весом более 20 Мб.** Если файл получается больше, заранее разделите его на несколько.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания будут сдавать устный зачет.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*


**Баллы за задание:**

* Задача 1 &mdash; 100 баллов
* Задача 2 &mdash; 50 баллов

Баллы учитываются в <b><font color="green">факультативной части</font></b> курса и не влияют на оценку по основной части.

In [5]:
# Bot check

# HW_ID: fpmi_ad6
# Бот проверит этот ID и предупредит, если случайно сдать что-то не то.

# Status: final
# Перед отправкой в финальном решении удали "not" в строчке выше.
# Так бот проверит, что ты отправляешь финальную версию, а не промежуточную.
# Никакие значения в этой ячейке не влияют на факт сдачи работы.

In [7]:
import time
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from string import punctuation
import matplotlib.pyplot as plt
from collections import Counter, OrderedDict
from IPython.display import clear_output
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torchtext
from torchtext.data.utils import get_tokenizer
import seaborn as sns
sns.set(palette="Set2")
sns.set_style("whitegrid")
import nltk
nltk.download("stopwords")
import os

device = "cuda"
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
print(device)

cuda


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/akruglov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Перед выполнением задания обязательно посмотрите <a href="https://miptstats.github.io/courses/ad_fivt/lecture6.pdf" target="_blank">презентацию</a>, [ноутбук](https://miptstats.github.io/courses/ad_fivt/nlp_sem.html) по RNN и [ноутбук](https://miptstats.github.io/courses/ad_fivt/nlp_sem_llama.html) по генерации текста.

---
### Задача 1. Предсказание заработной платы


Последовал рекомендации и прикрепил отдельным файлом

---
### Задача 2. Использование большой языковой модели


Ваша задача &mdash; применить <a target="_blank" href="https://llama-2.ai/">Llama 2</a> к предыдущей задаче и заставить ее предсказать заработную плату по текстовому описанию. Существует мало русскоязычных моделей, способных решить такую задачу и помещающихся в <a target="_blank" href="https://colab.research.google.com/">Colab</a>, поэтому вам придется писать запросы на английском!

In [8]:
# !pip3 install -q bitsandbytes==0.41.1 transformers==4.34.1 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 auto-gptq==0.4.2  > null
import transformers
import bitsandbytes as bnb
assert torch.cuda.is_available(), "для этой части понадобится GPU"

Дисклеймер: *использовать LLM в рамках ограничений Colab хоть и возможно, но очень трудно.*

> Так как процесс генерации очень хрупкий и может сломаться при любом неверном движении, мы настоятельно рекомендуем **сохранить результаты предыдущего задания в отдельном файле** и приступать к этой задаче в самом конце. Решения задач можно сдавать в бот разными файлами.

> Если у вас возникла ошибка `Out of memory`, перезапустите ноутбук и попробуйте снова. Да, языковые модели без своей GPU &mdash; это тяжело...





От вас требуется подобрать такой промпт, который приводил бы к нужному выводу модели. Можете экспериментировать с параметрами генерации, список которых можно посмотреть в <a target="_blank" href="https://github.com/huggingface/transformers/blob/c4d4e8bdbd25d9463d41de6398940329c89b7fb6/src/transformers/generation_utils.py#L101">исходнике</a>.

Существуют различные техники, которые могут помочь вам "разговорить" языковую модель. О многих из них вы сможете прочитать <a target="_blank" href="https://docs.mistral.ai/guides/prompting-capabilities/">здесь</a>. В нашем случае полезным может быть метод Few-Shot Learning, который заключается в предоставлении модели нескольких примеров.

In [28]:
model_name = "TheBloke/Llama-2-13B-GPTQ"  # Загружаем Llama токенизатор и саму модель Llama
tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_state_dict=True,
)

Прогоним ровно на примерах из конца 1й задачи, чтобы было с чем сравнивать (скопировал с одного из запусков)

In [29]:
descriptions = [
    'Are looking next move Gas Engineering career An excellent position arisen join prestigious Facilities Management company Maintenance Engineer The client requires candidates Fife Edinburgh locations carry maintenance coverage across Central Scotland Dundee You background Gas Installation Servicing Maintenance Repair Candidates required excellent communication skills role client facing attending customer sites This role report directly Contracts Manager require candidates direct liaison management client Essential qualification Maintenance Engineering position Commercial Gas Safe Domestic Gas Safe Testing Candidates following also highly desirable 17th Edition This fantastic opportunity right individual progress career enter solid organisation As mobile role driving licence essential This permanent Maintenance Engineer role covering Central Scotland comes hourly rate /hour hr week Please send CV today RECRUITING NOW This job originally posted',
    'Our client plus bedroom top end luxury star hotel substantial food beverage outlets An individual possesses background large scale operations able manage motivate lead big team required join management modern stylish hotel operation You innovative approach expected challenge existing methods carrying tasks The remit covers busy food drink outlets employ staff per day It include banqueting Reporting Food Beverage Director essential successful candidate progressive experience volume catering operations operating top end market ambitious develop career upwards top end hotel operations OTE /****k',
    'Experienced Accounts Administrator required reputable company based Somerset This initially offered years contract possibility extension 14 months This full time position working either 8.30am   4 pm 9   5 pm Monday Friday salary   PA DOE Seeking experienced Accounts Administrator join dynamic friendly highly successful organisation Duties role include   Ensuring customer delivery paid time   Processing payments   Reconciling supplier statements Purchase Ledger   Sales Ledger   Sending remittance general suppliers   Issuing reconciling petty cash   Processing sales invoices   Some payroll duties   Ad hoc duties support team The successful applicant competent user computerised accounts software Microsoft Word Excel ability work deadlines good accuracy skills To apply role please online contact Office Angels equal opportunities employer This job originally posted',
    'FINTEC recruit seeking electrical Designer projects This working client defence sector Experience ship essential Immediate start Experience GRP Composite structures You must possess MS office skills Applicants must security clearance possible application C',
    'The ideal candidate mechanical engineering background preferably project engineering CAD design experience Computer literate good working knowledge word excel Excellent communication skills imperative The working hours Mon   Fri 9 acting Employment Agency relation vacancy'
]
true_targets = [28800.0, 35000.0, 16000.0, 42240.0, 40000.0]

Я слышал, что использование в промпте вежливых слов вроде "please" и оборотов наподобие "it is important for my work" повышает качество работы генеративных текстовых моделей. Это явление можно объяснить тем, что такие модели (особенно, чат-боты) могут обучаться на человеческих переписках, в которых вежливость и апелляция к важности задачи помогают добиться желаемого ответа от собеседника. Однако, в этой задаче у меня такой подход не сработал (хотя он работал с другими моделями в режиме чат-бота)  

Следующий подход, который я поробовал, заключался в том, чтобы стилизовать аутпут под объявление о вакансии. Далее модель использовалась, чтобы дополнить объявление информацией о зарплате. Это уже сработало лучше  

In [44]:
for desc, target in zip(descriptions, true_targets):
    prompt = f"NOW HIRING! JOB DESCRIPTION:\n'{desc[:500]}...'\nSALARY in $:"  # после 'SALARY in $' будет предикт модели
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generate_ids = model.generate(inputs.input_ids, max_length=136)
    print(tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])
    print(f'True target: {target}\n--------------------')

NOW HIRING! JOB DESCRIPTION:
'Are looking next move Gas Engineering career An excellent position arisen join prestigious Facilities Management company Maintenance Engineer The client requires candidates Fife Edinburgh locations carry maintenance coverage across Central Scotland Dundee You background Gas Installation Servicing Maintenance Repair Candidates required excellent communication skills role client facing attending customer sites This role report directly Contracts Manager require candidates direct liaison management...'
SALARY in $: 30000 - 35000
SALARY in £: 20000 - 2
True target: 28800.0
--------------------
NOW HIRING! JOB DESCRIPTION:
'Our client plus bedroom top end luxury star hotel substantial food beverage outlets An individual possesses background large scale operations able manage motivate lead big team required join management modern stylish hotel operation You innovative approach expected challenge existing methods carrying tasks The remit covers busy food drink ou

Если почувствуете, что у вас не получается добиться от модели желаемого, вы можете попробовать воспользоваться любой другой языковой моделью на ваш выбор, но тогда вы получите **не больше 30 баллов**.

В случае использования другой модели предоставьте скриншот ответа LLM и промпт, который вы использовали (в текстовом формате). Его можно вставить прямо в ноутбук или прислать в бот отдельным файлом.

**Вывод:**  
В рамках задачи удалось подобрать промпт, с помощью которого модель выдает более-менее осмысленный ответ. Использовать такой подход на практике, однако, рискованно (качество низкое, работа нестабильная). По-моему, лучше воспользоваться моделью из первой задачи.   
Большие языковые модели настолько сложно устроены, что даже для того, чтобы использовать их, необходимо овладеть искусством промпт-инжиниринга. Интересно, что часто непонятно, как подбирать оптимальный промпт: иногда выигрывают совершенно удивительные стратегии  

---
*Если хотите отточить навык написания промптов &mdash; **prompt-engineering** &mdash; можете попробовать сыграть в <a target="_blank" href="https://gandalf.lakera.ai/">игру</a>. В ней нужно заставить Гендальфа выдать пароль. Со временем уровни становятся все сложнее и требуют более хитрых приемов.*